In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os
from collections import Counter
from imblearn.over_sampling import SMOTE

In [4]:
from feature_selection import forwardFeatureSelection

from NonLinCFA import NonLinCFA
from aux_GenLinCFA import prepare_target_binary

from aux_NonLinCFA import *

In [5]:
cd ..

/content/drive/MyDrive/Colab Notebooks


### Multi task functions

In [ ]:
# for binary classification 

from sklearn.metrics import accuracy_score
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())
    
    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

In [6]:
# for linear regression 

def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    r2_trainval_cluster = []
    r2_test_cluster = []
    r2_val_cluster = []

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_test = LinearRegression()
    model_test.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())
    model_val = LinearRegression()
    model_val.fit(clusterdf_train_withClass.values, targets_df_train_unfolded.values.ravel())

    for basin in clust_basins:
      print(basin)
      res_test = model_test.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
      res_trainval = model_test.predict(pd.concat([clusterdf_train_withClass.loc[clusterdf_train_withClass[basin]==1], 
                                                   clusterdf_val_withClass.loc[clusterdf_val_withClass[basin]==1]]).values)
      res_val = model_val.predict(clusterdf_val_withClass.loc[clusterdf_val_withClass[basin]==1].values)
      r2_trainval = r2_score(pd.concat([targets_df_train[basin], targets_df_val[basin]], axis =0).values.ravel(), 
                             res_trainval)
      r2_trainval_cluster.append(r2_trainval)
      print("Score in trainval: ", round(r2_trainval,2))
      r2_test = r2_score(targets_df_test[basin].values.ravel(), res_test)
      r2_test_cluster.append(r2_test)
      print("Score in test: ", round(r2_test,2))
      r2_val = r2_score(targets_df_val[basin].values.ravel(), res_val)
      r2_val_cluster.append(r2_val)
      print("Score in val: ", round(r2_val,2))
    return r2_trainval_cluster, r2_test_cluster, r2_val_cluster

#Shuffle choice linear regression

In [7]:
def compute_scores_best_shuffles(shuffle_path, normal_path, basins, singleTask, best_shuffles):

  targets_df_trainVal = pd.DataFrame()
  targets_df_test = pd.DataFrame()
  targets_df_train = pd.DataFrame()
  targets_df_val = pd.DataFrame()
  best5_CMI_fulldf_train = pd.DataFrame()
  best5_CMI_fulldf_val = pd.DataFrame()
  best5_CMI_fulldf_test = pd.DataFrame()

  for basin in basins: 
    path_targets = "./csv/"
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                    max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std
    targets_df_test[basin] = target_df_test.mean_std

    best_shuffle_id = best_shuffles[basin]

    if (best_shuffle_id != 5):
      train_temp = pd.read_csv(shuffle_path+basin+'_' + str(best_shuffle_id) + '_nonLinCFA_best5_CMI_train.csv')
      val_temp = pd.read_csv(shuffle_path+basin+'_' + str(best_shuffle_id) + '_nonLinCFA_best5_CMI_val.csv')
      test_temp = pd.read_csv(shuffle_path+basin+'_' + str(best_shuffle_id) + '_nonLinCFA_best5_CMI_test.csv')
    else:
      train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
      val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
      test_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_test.csv')

    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    
  if singleTask:
    best5_CMI_train_val = pd.concat([train_temp,val_temp])
    lin_regr_test = LinearRegression()
    lin_regr_test.fit(best5_CMI_train_val, targets_df_trainVal[basin])

    lin_regr_val = LinearRegression()
    lin_regr_val.fit(train_temp, targets_df_train[basin])

    r2_trainval = lin_regr_test.score(best5_CMI_train_val, targets_df_trainVal[basin])
    print("Score in trainval: ", round(r2_trainval,2))
    r2_test = lin_regr_test.score(test_temp, targets_df_test[basin])
    print("Score in test: ", round(r2_test,2))
    r2_val = lin_regr_val.score(val_temp, targets_df_val[basin])
    print("Score in val: ", round(r2_val,2))

  else:
    print("\n---- MultiTask regression scores: ----\n")
    MTL_scores(clust_basins=basins, df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, 
               targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

## Cross-validation for time series to choose the best shuffle 3 splits, training set bf and after valid

In [ ]:
i = 0
basin = 'Adda'
shuffle_path = './NonLinCFA/temp_prec_shuffle/'
normal_path = './NonLinCFA/temp_prec/'
### CMI best5 features
best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
if i!=5:
  print("Cross validation for time series of ", basin, " for shuffle ", str(i))
  train_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
  val_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
else:
  print("Cross validation for time series of ", basin, " without shuffle")
  train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
  val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')

# continuous target
path_targets = "./csv/"
targets_df_trainVal = pd.DataFrame()
target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
targets_df_trainVal[basin] = target_df_trainVal.mean_std

X = pd.concat([train_temp, val_temp])
X.reset_index(inplace=True, drop=True)
y = targets_df_trainVal[basin]
y.reset_index(inplace=True, drop=True)
r2 = []
# num of 8 days in 2 years are around 92
tscv = TimeSeriesSplit(max_train_size=None, n_splits=3, test_size = 184)
for train_index, valid_index in tscv.split(X):
  # se valid_index non contiene l'ultimo indice, aggiungi a train_index gli indici dall'ultimo di valid_index fino alla fine
  if (valid_index[-1] != len(X)-1):
    train_index = np.concatenate((train_index, np.array(range(valid_index[-1] + 1, len(X)))))
  print("Train length : ", len(train_index), ", validation length : ", len(valid_index))
  print(valid_index)
  print(train_index)
  X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
  y_train, y_valid = y[train_index], y[valid_index]

Cross validation for time series of  Adda  for shuffle  0
Train length :  455 , validation length :  184
[ 87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104
 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122
 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158
 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176
 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194
 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212
 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230
 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248
 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266
 267 268 269 270]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
 

### 4 years of validation set

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, shuffle_path, normal_path):
  best_shuffles = {}

  for basin in basins :
    mean_r2_shuffles = []
    for i in range(0,6):

      ### CMI best5 features
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      if i!=5:
        print("Cross validation for time series of ", basin, " for shuffle ", str(i))
        train_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
      else:
        print("Cross validation for time series of ", basin, " without shuffle")
        train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

      X = pd.concat([train_temp, val_temp])
      X.reset_index(inplace=True, drop=True)
      y = targets_df_trainVal[basin]
      y.reset_index(inplace=True, drop=True)
      r2 = []
      # num of 8 days in 2 years are around 92
      tscv = TimeSeriesSplit(max_train_size=None, n_splits=3, test_size = 184)
      for train_index, valid_index in tscv.split(X):
        # se valid_index non contiene l'ultimo indice, aggiungi a train_index gli indici dall'ultimo di valid_index fino alla fine
        if (valid_index[-1] != len(X)-1):
          train_index = np.concatenate((train_index, np.array(range(valid_index[-1] + 1, len(X)))))
        print("Train length : ", len(train_index), ", validation length : ", len(valid_index))
        X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
        y_train, y_valid = y[train_index], y[valid_index]
        lin_regr = LinearRegression()
        lin_regr.fit(X_train, y_train)
        r2.append(lin_regr.score(X_valid, y_valid))
        print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train, y_train),3))
        print("Valid R2 linear regression CMI best 5: ", round(lin_regr.score(X_valid, y_valid),3))

      mean_r2_shuffles.append(np.mean(r2))
      if i!=5:
        print("Mean R2 for basin ", basin ," for shuffle ", str(i), " : ", str(round(np.mean(r2), 3)), "\n")
      else:
        print("Mean R2 for basin ", basin ," without shuffle: ", str(round(np.mean(r2), 3)), "\n")

    best_shuffle_id = mean_r2_shuffles.index(max(mean_r2_shuffles)) 
    if best_shuffle_id != 5:
      print("--- BEST SHUFFLE FOR ", basin, " : ", str(best_shuffle_id), "---\n")
    else:
      print("--- BEST SHUFFLE FOR ", basin, " : no shuffle---\n")
    best_shuffles[basin] = best_shuffle_id
  return best_shuffles

In [ ]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/', './NonLinCFA/temp_prec/')

Cross validation for time series of  Adda  for shuffle  0
Train length :  455 , validation length :  184
Train R2 linear regression CMI best 5:  0.203
Valid R2 linear regression CMI best 5:  -0.066
Train length :  455 , validation length :  184
Train R2 linear regression CMI best 5:  0.203
Valid R2 linear regression CMI best 5:  0.054
Train length :  455 , validation length :  184
Train R2 linear regression CMI best 5:  0.111
Valid R2 linear regression CMI best 5:  0.233
Mean R2 for basin  Adda  for shuffle  0  :  0.074 

Cross validation for time series of  Adda  for shuffle  1
Train length :  455 , validation length :  184
Train R2 linear regression CMI best 5:  0.229
Valid R2 linear regression CMI best 5:  -0.277
Train length :  455 , validation length :  184
Train R2 linear regression CMI best 5:  0.162
Valid R2 linear regression CMI best 5:  0.089
Train length :  455 , validation length :  184
Train R2 linear regression CMI best 5:  0.092
Valid R2 linear regression CMI best 5:  0.

In [ ]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  print(basin)
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles)

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

Adda
Score in trainval:  0.18
Score in test:  0.2
Score in val:  0.2
Dora
Score in trainval:  0.12
Score in test:  -0.43
Score in val:  0.11
Emiliani1
Score in trainval:  0.37
Score in test:  0.25
Score in val:  0.39
Emiliani2
Score in trainval:  0.24
Score in test:  0.14
Score in val:  0.26
Garda_Mincio
Score in trainval:  0.22
Score in test:  0.2
Score in val:  0.06
Lambro_Olona
Score in trainval:  0.24
Score in test:  0.16
Score in val:  0.28
Oglio_Iseo
Score in trainval:  0.18
Score in test:  0.21
Score in val:  0.19
Piemonte_Nord
Score in trainval:  0.22
Score in test:  0.04
Score in val:  0.11
Piemonte_Sud
Score in trainval:  0.18
Score in test:  0.08
Score in val:  0.16
Ticino
Score in trainval:  0.19
Score in test:  0.17
Score in val:  0.21

---- MultiTask regression scores: ----

Emiliani1
Score in trainval:  0.39
Score in test:  0.26
Score in val:  0.34
Emiliani2
Score in trainval:  0.31
Score in test:  0.16
Score in val:  0.3
Garda_Mi

### 2 years of validation set

In [15]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, shuffle_path, normal_path):
  best_shuffles = {}

  for basin in basins :
    mean_r2_shuffles = []
    for i in range(0,6):

      ### CMI best5 features
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      if i!=5:
        print("Cross validation for time series of ", basin, " for shuffle ", str(i))
        train_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
      else:
        print("Cross validation for time series of ", basin, " without shuffle")
        train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

      X = pd.concat([train_temp, val_temp])
      X.reset_index(inplace=True, drop=True)
      y = targets_df_trainVal[basin]
      y.reset_index(inplace=True, drop=True)
      r2 = []
      # num of 8 days in 2 years are around 92
      tscv = TimeSeriesSplit(max_train_size=None, n_splits=3, test_size = 92)
      for train_index, valid_index in tscv.split(X):
        # se valid_index non contiene l'ultimo indice, aggiungi a train_index gli indici dall'ultimo di valid_index fino alla fine
        if (valid_index[-1] != len(X)-1):
          train_index = np.concatenate((train_index, np.array(range(valid_index[-1] + 1, len(X)))))
        print("Train length : ", len(train_index), ", validation length : ", len(valid_index))
        X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
        y_train, y_valid = y[train_index], y[valid_index]
        lin_regr = LinearRegression()
        lin_regr.fit(X_train, y_train)
        r2.append(lin_regr.score(X_valid, y_valid))
        print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train, y_train),3))
        print("Valid R2 linear regression CMI best 5: ", round(lin_regr.score(X_valid, y_valid),3))

      mean_r2_shuffles.append(np.mean(r2))
      if i!=5:
        print("Mean R2 for basin ", basin ," for shuffle ", str(i), " : ", str(round(np.mean(r2), 3)), "\n")
      else:
        print("Mean R2 for basin ", basin ," without shuffle: ", str(round(np.mean(r2), 3)), "\n")

    best_shuffle_id = mean_r2_shuffles.index(max(mean_r2_shuffles)) 
    if best_shuffle_id != 5:
      print("--- BEST SHUFFLE FOR ", basin, " : ", str(best_shuffle_id), "---\n")
    else:
      print("--- BEST SHUFFLE FOR ", basin, " : no shuffle---\n")
    best_shuffles[basin] = best_shuffle_id
  return best_shuffles

In [16]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/', './NonLinCFA/temp_prec/')

Cross validation for time series of  Adda  for shuffle  0
Train length :  547 , validation length :  92
Train R2 linear regression CMI best 5:  0.198
Valid R2 linear regression CMI best 5:  -0.027
Train length :  547 , validation length :  92
Train R2 linear regression CMI best 5:  0.14
Valid R2 linear regression CMI best 5:  0.322
Train length :  547 , validation length :  92
Train R2 linear regression CMI best 5:  0.141
Valid R2 linear regression CMI best 5:  0.176
Mean R2 for basin  Adda  for shuffle  0  :  0.157 

Cross validation for time series of  Adda  for shuffle  1
Train length :  547 , validation length :  92
Train R2 linear regression CMI best 5:  0.168
Valid R2 linear regression CMI best 5:  0.011
Train length :  547 , validation length :  92
Train R2 linear regression CMI best 5:  0.118
Valid R2 linear regression CMI best 5:  0.333
Train length :  547 , validation length :  92
Train R2 linear regression CMI best 5:  0.13
Valid R2 linear regression CMI best 5:  0.116
Mean 

In [17]:
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  print(basin)
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles)

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

Adda
Score in trainval:  0.18
Score in test:  0.24
Score in val:  0.26
Dora
Score in trainval:  0.12
Score in test:  -0.43
Score in val:  0.04
Emiliani1
Score in trainval:  0.37
Score in test:  0.27
Score in val:  0.42
Emiliani2
Score in trainval:  0.23
Score in test:  0.25
Score in val:  0.32
Garda_Mincio
Score in trainval:  0.22
Score in test:  0.16
Score in val:  0.1
Lambro_Olona
Score in trainval:  0.24
Score in test:  0.24
Score in val:  0.24
Oglio_Iseo
Score in trainval:  0.17
Score in test:  0.26
Score in val:  0.12
Piemonte_Nord
Score in trainval:  0.22
Score in test:  0.04
Score in val:  0.11
Piemonte_Sud
Score in trainval:  0.18
Score in test:  0.08
Score in val:  0.16
Ticino
Score in trainval:  0.19
Score in test:  0.18
Score in val:  0.26

---- MultiTask regression scores: ----

Emiliani1
Score in trainval:  0.4
Score in test:  0.31
Score in val:  0.29
Emiliani2
Score in trainval:  0.32
Score in test:  0.21
Score in val:  0.24
Garda_

## Cross-validation for time series to choose the best shuffle (regression) 3 splits

In [22]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, shuffle_path, normal_path):
  best_shuffles = {}

  for basin in basins :
    mean_r2_shuffles = []
    for i in range(0,6):

      ### CMI best5 features
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      if i!=5:
        print("Cross validation for time series of ", basin, " for shuffle ", str(i))
        train_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
      else:
        print("Cross validation for time series of ", basin, " without shuffle")
        train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

      X = pd.concat([train_temp, val_temp])
      X.reset_index(inplace=True, drop=True)
      y = targets_df_trainVal[basin]
      y.reset_index(inplace=True, drop=True)
      r2 = []
      # num of 8 days in 2 years are around 92
      tscv = TimeSeriesSplit(max_train_size=None, n_splits=3, test_size = 92)
      for train_index, valid_index in tscv.split(X):
        print("Train length : ", len(train_index), ", validation length : ", len(valid_index)) 
        X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
        y_train, y_valid = y[train_index], y[valid_index]
        lin_regr = LinearRegression()
        lin_regr.fit(X_train, y_train)
        r2.append(lin_regr.score(X_valid, y_valid))
        print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train, y_train),3))
        print("Valid R2 linear regression CMI best 5: ", round(lin_regr.score(X_valid, y_valid),3))

      mean_r2_shuffles.append(np.mean(r2))
      if i!=5:
        print("Mean R2 for basin ", basin ," for shuffle ", str(i), " : ", str(round(np.mean(r2), 3)), "\n")
      else:
        print("Mean R2 for basin ", basin ," without shuffle: ", str(round(np.mean(r2), 3)), "\n")

    best_shuffle_id = mean_r2_shuffles.index(max(mean_r2_shuffles)) 
    if best_shuffle_id != 5:
      print("--- BEST SHUFFLE FOR ", basin, " : ", str(best_shuffle_id), "---\n")
    else:
      print("--- BEST SHUFFLE FOR ", basin, " : no shuffle---\n")
    best_shuffles[basin] = best_shuffle_id
  return best_shuffles

In [23]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/', './NonLinCFA/temp_prec/')

Cross validation for time series of  Adda  for shuffle  0
Train length :  271 , validation length :  92
Train R2 linear regression CMI best 5:  0.13
Valid R2 linear regression CMI best 5:  0.14
Train length :  363 , validation length :  92
Train R2 linear regression CMI best 5:  0.135
Valid R2 linear regression CMI best 5:  0.02
Train length :  455 , validation length :  92
Train R2 linear regression CMI best 5:  0.111
Valid R2 linear regression CMI best 5:  0.289
Train length :  547 , validation length :  92
Train R2 linear regression CMI best 5:  0.141
Valid R2 linear regression CMI best 5:  0.176
Mean R2 for basin  Adda  for shuffle  0  :  0.156 

Cross validation for time series of  Adda  for shuffle  1
Train length :  271 , validation length :  92
Train R2 linear regression CMI best 5:  0.117
Valid R2 linear regression CMI best 5:  -0.132
Train length :  363 , validation length :  92
Train R2 linear regression CMI best 5:  0.116
Valid R2 linear regression CMI best 5:  -0.046
Train

In [24]:
basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio']
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  print(basin)
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles)

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

Emiliani1
Score in trainval:  0.37
Score in test:  0.25
Score in val:  0.39
Emiliani2
Score in trainval:  0.23
Score in test:  0.25
Score in val:  0.32
Garda_Mincio
Score in trainval:  0.22
Score in test:  0.2
Score in val:  0.06

---- MultiTask regression scores: ----

Emiliani1
Score in trainval:  0.4
Score in test:  0.3
Score in val:  0.26
Emiliani2
Score in trainval:  0.32
Score in test:  0.2
Score in val:  0.18
Garda_Mincio
Score in trainval:  0.3
Score in test:  0.21
Score in val:  0.01

---- MultiTask regression scores: ----

Adda
Score in trainval:  0.23
Score in test:  0.18
Score in val:  0.15
Lambro_Olona
Score in trainval:  0.27
Score in test:  0.16
Score in val:  0.15
Oglio_Iseo
Score in trainval:  0.23
Score in test:  0.22
Score in val:  0.15
Ticino
Score in trainval:  0.24
Score in test:  0.18
Score in val:  0.16

---- MultiTask regression scores: ----

Dora
Score in trainval:  0.19
Score in test:  -0.52
Score in val:  0.17
Piemont

## Cross-validation for time series to choose the best shuffle (regression) 2 splits fixed training set

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, path):
  best_shuffles = {}

  for basin in basins :
    mean_r2_shuffles = []
    for i in range(0,5):
      print("Cross validation for time series of ", basin, " for shuffle ", str(i))

      ### CMI best5 features
      path_features = path
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      train_temp = pd.read_csv(path_features+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
      val_temp = pd.read_csv(path_features+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

      X = pd.concat([train_temp, val_temp])
      X.reset_index(inplace=True, drop=True)
      y = targets_df_trainVal[basin]
      y.reset_index(inplace=True, drop=True)
      r2 = []
      # num of 8 days in 2 years are around 92
      tscv = TimeSeriesSplit(max_train_size=455, n_splits=2, test_size = 92)
      for train_index, valid_index in tscv.split(X):
        print("Train length : ", len(train_index), ", validation length : ", len(valid_index)) 
        X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
        y_train, y_valid = y[train_index], y[valid_index]
        lin_regr = LinearRegression()
        lin_regr.fit(X_train, y_train)
        r2.append(lin_regr.score(X_valid, y_valid))
        print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train, y_train),3))
        print("Valid R2 linear regression CMI best 5: ", round(lin_regr.score(X_valid, y_valid),3))

      mean_r2_shuffles.append(np.mean(r2))
      print("Mean R2 for basin ", basin ," for shuffle ", str(i), " : ", str(round(np.mean(r2), 3)), "\n")
    print("--- BEST SHUFFLE FOR ", basin, " : ", mean_r2_shuffles.index(max(mean_r2_shuffles)), "---\n")
    best_shuffles[basin] = mean_r2_shuffles.index(max(mean_r2_shuffles))
  return best_shuffles

In [ ]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/')

Cross validation for time series of  Adda  for shuffle  0
Train length :  455 , validation length :  92
Train R2 linear regression CMI best 5:  0.111
Valid R2 linear regression CMI best 5:  0.289
Train length :  455 , validation length :  92
Train R2 linear regression CMI best 5:  0.137
Valid R2 linear regression CMI best 5:  0.168
Mean R2 for basin  Adda  for shuffle  0  :  0.228 

Cross validation for time series of  Adda  for shuffle  1
Train length :  455 , validation length :  92
Train R2 linear regression CMI best 5:  0.092
Valid R2 linear regression CMI best 5:  0.295
Train length :  455 , validation length :  92
Train R2 linear regression CMI best 5:  0.149
Valid R2 linear regression CMI best 5:  0.046
Mean R2 for basin  Adda  for shuffle  1  :  0.17 

Cross validation for time series of  Adda  for shuffle  2
Train length :  455 , validation length :  92
Train R2 linear regression CMI best 5:  0.102
Valid R2 linear regression CMI best 5:  0.321
Train length :  455 , validation 

In [ ]:
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  print(basin)
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles)

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

Adda
Score in trainval:  0.18
Score in test:  0.24
Score in val:  0.26
Dora
Score in trainval:  0.12
Score in test:  -0.43
Score in val:  0.04
Emiliani1
Score in trainval:  0.36
Score in test:  0.24
Score in val:  0.41
Emiliani2
Score in trainval:  0.23
Score in test:  0.25
Score in val:  0.32
Garda_Mincio
Score in trainval:  0.22
Score in test:  0.16
Score in val:  0.1
Lambro_Olona
Score in trainval:  0.21
Score in test:  0.29
Score in val:  0.26
Oglio_Iseo
Score in trainval:  0.18
Score in test:  0.21
Score in val:  0.19
Piemonte_Nord
Score in trainval:  0.21
Score in test:  0.17
Score in val:  0.14
Piemonte_Sud
Score in trainval:  0.18
Score in test:  0.08
Score in val:  0.16
Ticino
Score in trainval:  0.19
Score in test:  0.18
Score in val:  0.26

---- MultiTask regression scores: ----

Emiliani1
Score in trainval:  0.4
Score in test:  0.28
Score in val:  0.36
Emiliani2
Score in trainval:  0.32
Score in test:  0.16
Score in val:  0.32
Garda_

##Fixed validation set

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, path):
  best_shuffles = {}

  for basin in basins :
    r2 = []
    for i in range(0,5):
      print("Cross validation for time series of ", basin, " for shuffle ", str(i))

      ### CMI best5 features
      path_features = path
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      training_df = pd.read_csv(path_features+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
      valid_df = pd.read_csv(path_features+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std
      lin_regr = LinearRegression()
      lin_regr.fit(training_df, target_df_train.mean_std)
      valid_score = lin_regr.score(valid_df, target_df_val.mean_std)
      r2.append(valid_score)
      print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(training_df, target_df_train.mean_std),3))
      print("Valid R2 linear regression CMI best 5: ", round(valid_score,3))

    print("\n--- BEST SHUFFLE FOR ", basin, " : ", r2.index(max(r2)), "---\n")
    best_shuffles[basin] = r2.index(max(r2))
  return best_shuffles

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, shuffle_path, normal_path):
  best_shuffles = {}

  for basin in basins :
    r2 = []
    for i in range(0,6):
      #print("Cross validation for time series of ", basin, " for shuffle ", str(i))

      ### CMI best5 features
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      if i!=5:
        print("Cross validation for time series of ", basin, " for shuffle ", str(i))
        training_df = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
        valid_df = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
      else:
        print("Cross validation for time series of ", basin, " without shuffle")
        training_df = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
        valid_df = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std
      lin_regr = LinearRegression()
      lin_regr.fit(training_df, target_df_train.mean_std)
      valid_score = lin_regr.score(valid_df, target_df_val.mean_std)
      r2.append(valid_score)
      print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(training_df, target_df_train.mean_std),3))
      print("Valid R2 linear regression CMI best 5: ", round(valid_score,3))

    best_shuffle_id = r2.index(max(r2))
    if best_shuffle_id != 5:
      print("--- BEST SHUFFLE FOR ", basin, " : ", str(best_shuffle_id), "---\n")
    else:
      print("--- BEST SHUFFLE FOR ", basin, " : no shuffle---\n")
    best_shuffles[basin] = best_shuffle_id

  return best_shuffles

In [ ]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Emiliani1','Emiliani2','Garda_Mincio']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/', './NonLinCFA/temp_prec/')

Cross validation for time series of  Emiliani1  for shuffle  0
Train R2 linear regression CMI best 5:  0.285
Valid R2 linear regression CMI best 5:  0.413
Cross validation for time series of  Emiliani1  for shuffle  1
Train R2 linear regression CMI best 5:  0.259
Valid R2 linear regression CMI best 5:  0.387
Cross validation for time series of  Emiliani1  for shuffle  2
Train R2 linear regression CMI best 5:  0.329
Valid R2 linear regression CMI best 5:  0.341
Cross validation for time series of  Emiliani1  for shuffle  3
Train R2 linear regression CMI best 5:  0.322
Valid R2 linear regression CMI best 5:  0.394
Cross validation for time series of  Emiliani1  for shuffle  4
Train R2 linear regression CMI best 5:  0.308
Valid R2 linear regression CMI best 5:  0.424
Cross validation for time series of  Emiliani1  without shuffle
Train R2 linear regression CMI best 5:  0.284
Valid R2 linear regression CMI best 5:  0.365
--- BEST SHUFFLE FOR  Emiliani1  :  4 ---

Cross validation for time 

In [ ]:
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  print(basin, " ", compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles))

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

Adda   0.24
Dora   -0.46
Emiliani1   0.27
Emiliani2   0.25
Garda_Mincio   0.16
Lambro_Olona   0.16
Oglio_Iseo   0.21
Piemonte_Nord   0.17
Piemonte_Sud   0.08
Ticino   0.18

---- MultiTask regression scores: ----

Emiliani1
0.30509691195851074
Emiliani2
0.2101604964780014
Garda_Mincio
0.2071635896010613

---- MultiTask regression scores: ----

Adda
0.1240154869793254
Lambro_Olona
0.15605859303391967
Oglio_Iseo
0.18626962945085734
Ticino
0.16445985016532882

---- MultiTask regression scores: ----

Dora
-0.5561985313739164
Piemonte_Sud
-0.320036023318538
Piemonte_Nord
-0.2831466061625523


#Shuffle choice binary classification

In [ ]:
def compute_scores_best_shuffles(path, basins, singleTask, best_shuffles):

  targets_df_trainVal = pd.DataFrame()
  targets_df_test = pd.DataFrame()
  targets_df_train = pd.DataFrame()
  targets_df_val = pd.DataFrame()
  best5_CMI_fulldf_train = pd.DataFrame()
  best5_CMI_fulldf_val = pd.DataFrame()
  best5_CMI_fulldf_test = pd.DataFrame()

  for basin in basins: 
    path_targets = "./csv/"
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                    max_test='2020-01-01', path=path_targets+basin+'.csv',
                                                                                    threshold = None, nopeaks = False)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std
    targets_df_test[basin] = target_df_test.mean_std

    train_temp = pd.read_csv(path+basin+'_' + str(best_shuffles[basin]) + '_genLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path+basin+'_' + str(best_shuffles[basin]) + '_genLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path+basin+'_' + str(best_shuffles[basin]) + '_genLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    
  if singleTask:
    best5_CMI_fulldf_train_val = pd.concat([train_temp,val_temp])
    log_regr = LogisticRegression(max_iter = 500)
    log_regr.fit(best5_CMI_fulldf_train_val, targets_df_trainVal[basin])
    return round(log_regr.score(test_temp, targets_df_test[basin]),2)
  else:
    print("\n---- MultiTask binary classification scores: ----\n")
    MTL_scores(clust_basins=basins, df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, 
               targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

## Cross-validation for time series to choose the best shuffle (binary classification)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, path):
  best_shuffles = {}

  for basin in basins :
    mean_accuracy_shuffles = []
    for i in range(0,5):
      print("Cross validation for time series of ", basin, " for shuffle ", str(i))

      ### CMI best5 features
      path_features = path
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      train_temp = pd.read_csv(path_features+basin+'_'+str(i)+'_genLinCFA_best5_CMI_train.csv')
      val_temp = pd.read_csv(path_features+basin+'_'+str(i)+'_genLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', 
                                                                                      threshold = None, nopeaks = False)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

      X = pd.concat([train_temp, val_temp])
      X.reset_index(inplace=True, drop=True)
      y = targets_df_trainVal[basin]
      y.reset_index(inplace=True, drop=True)
      accuracy = []
      # num of 8 days in 2 years are around 92
      tscv = TimeSeriesSplit(max_train_size=None, n_splits=3, test_size = 92)
      for train_index, valid_index in tscv.split(X):
        print("Train length : ", len(train_index), ", validation length : ", len(valid_index)) 
        X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
        y_train, y_valid = y[train_index], y[valid_index]
        log_regr = LogisticRegression(max_iter = 500)
        log_regr.fit(X_train, y_train)
        accuracy.append(log_regr.score(X_valid, y_valid))
        print("Train accuracy binary classification CMI best 5: ", round(log_regr.score(X_train, y_train),3))
        print("Valid accuracy binary classification CMI best 5: ", round(log_regr.score(X_valid, y_valid),3))

      mean_accuracy_shuffles.append(np.mean(accuracy))
      print("Mean accuracy for basin ", basin ," for mean_accuracy_shuffles ", str(i), " : ", str(round(np.mean(accuracy), 3)), "\n")
    print("--- BEST SHUFFLE FOR ", basin, " : ", mean_accuracy_shuffles.index(max(mean_accuracy_shuffles)), "---\n")
    best_shuffles[basin] = mean_accuracy_shuffles.index(max(mean_accuracy_shuffles))
  return best_shuffles

In [ ]:
features_path = './GenLinCFA/temp_prec_shuffle/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './GenLinCFA/temp_prec_shuffle/')

Cross validation for time series of  Adda  for shuffle  0
Train length :  363 , validation length :  92
Train accuracy binary classification CMI best 5:  0.581
Valid accuracy binary classification CMI best 5:  0.609
Train length :  455 , validation length :  92
Train accuracy binary classification CMI best 5:  0.578
Valid accuracy binary classification CMI best 5:  0.62
Train length :  547 , validation length :  92
Train accuracy binary classification CMI best 5:  0.576
Valid accuracy binary classification CMI best 5:  0.489
Mean accuracy for basin  Adda  for shuffle  0  :  0.572 

Cross validation for time series of  Adda  for shuffle  1
Train length :  363 , validation length :  92
Train accuracy binary classification CMI best 5:  0.562
Valid accuracy binary classification CMI best 5:  0.598
Train length :  455 , validation length :  92
Train accuracy binary classification CMI best 5:  0.569
Valid accuracy binary classification CMI best 5:  0.598
Train length :  547 , validation leng

In [ ]:
print("---- SingleTask binary classification scores: ----\n")
for basin in basins: 
  print(basin, " ", compute_scores_best_shuffles(features_path, [basin], True, best_shuffles))

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, cluster, False, best_shuffles)

---- SingleTask binary classification scores: ----

Adda   0.54
Dora   0.54
Emiliani1   0.69
Emiliani2   0.7
Garda_Mincio   0.71
Lambro_Olona   0.7
Oglio_Iseo   0.7
Piemonte_Nord   0.68
Piemonte_Sud   0.6
Ticino   0.67

---- MultiTask binary classification scores: ----

Emiliani1
0.7149122807017544
Emiliani2
0.7456140350877193
Garda_Mincio
0.7456140350877193

---- MultiTask binary classification scores: ----

Adda
0.6535087719298246
Lambro_Olona
0.6885964912280702
Oglio_Iseo
0.6578947368421053
Ticino
0.6885964912280702

---- MultiTask binary classification scores: ----

Dora
0.5570175438596491
Piemonte_Sud
0.5964912280701754
Piemonte_Nord
0.6140350877192983


## Cross-validation for time series to choose the best shuffle (binary classification) 2 splits fixed training set

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, path):
  best_shuffles = {}

  for basin in basins :
    mean_accuracy_shuffles = []
    for i in range(0,5):
      print("Cross validation for time series of ", basin, " for shuffle ", str(i))

      ### CMI best5 features
      path_features = path
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      train_temp = pd.read_csv(path_features+basin+'_'+str(i)+'_genLinCFA_best5_CMI_train.csv')
      val_temp = pd.read_csv(path_features+basin+'_'+str(i)+'_genLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', 
                                                                                      threshold = None, nopeaks = False)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

      X = pd.concat([train_temp, val_temp])
      X.reset_index(inplace=True, drop=True)
      y = targets_df_trainVal[basin]
      y.reset_index(inplace=True, drop=True)
      accuracy = []
      # num of 8 days in 2 years are around 92
      tscv = TimeSeriesSplit(max_train_size=455, n_splits=2, test_size = 92)
      for train_index, valid_index in tscv.split(X):
        print("Train length : ", len(train_index), ", validation length : ", len(valid_index)) 
        X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
        y_train, y_valid = y[train_index], y[valid_index]
        log_regr = LogisticRegression(max_iter = 500)
        log_regr.fit(X_train, y_train)
        accuracy.append(log_regr.score(X_valid, y_valid))
        print("Train accuracy binary classification CMI best 5: ", round(log_regr.score(X_train, y_train),3))
        print("Valid accuracy binary classification CMI best 5: ", round(log_regr.score(X_valid, y_valid),3))

      mean_accuracy_shuffles.append(np.mean(accuracy))
      print("Mean accuracy for basin ", basin ," for mean_accuracy_shuffles ", str(i), " : ", str(round(np.mean(accuracy), 3)), "\n")
    print("--- BEST SHUFFLE FOR ", basin, " : ", mean_accuracy_shuffles.index(max(mean_accuracy_shuffles)), "---\n")
    best_shuffles[basin] = mean_accuracy_shuffles.index(max(mean_accuracy_shuffles))
  return best_shuffles

In [ ]:
features_path = './GenLinCFA/temp_prec_shuffle/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './GenLinCFA/temp_prec_shuffle/')

Cross validation for time series of  Adda  for shuffle  0
Train length :  455 , validation length :  92
Train accuracy binary classification CMI best 5:  0.578
Valid accuracy binary classification CMI best 5:  0.62
Train length :  455 , validation length :  92
Train accuracy binary classification CMI best 5:  0.591
Valid accuracy binary classification CMI best 5:  0.489
Mean accuracy for basin  Adda  for mean_accuracy_shuffles  0  :  0.554 

Cross validation for time series of  Adda  for shuffle  1
Train length :  455 , validation length :  92
Train accuracy binary classification CMI best 5:  0.569
Valid accuracy binary classification CMI best 5:  0.598
Train length :  455 , validation length :  92
Train accuracy binary classification CMI best 5:  0.565
Valid accuracy binary classification CMI best 5:  0.522
Mean accuracy for basin  Adda  for mean_accuracy_shuffles  1  :  0.56 

Cross validation for time series of  Adda  for shuffle  2
Train length :  455 , validation length :  92
Trai

In [ ]:
print("---- SingleTask binary classification scores: ----\n")
for basin in basins: 
  print(basin, " ", compute_scores_best_shuffles(features_path, [basin], True, best_shuffles))

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, cluster, False, best_shuffles)

---- SingleTask binary classification scores: ----

Adda   0.55
Dora   0.56
Emiliani1   0.71
Emiliani2   0.7
Garda_Mincio   0.71
Lambro_Olona   0.7
Oglio_Iseo   0.7
Piemonte_Nord   0.63
Piemonte_Sud   0.6
Ticino   0.67

---- MultiTask binary classification scores: ----

Emiliani1
0.706140350877193
Emiliani2
0.75
Garda_Mincio
0.7675438596491229

---- MultiTask binary classification scores: ----

Adda
0.6491228070175439
Lambro_Olona
0.6842105263157895
Oglio_Iseo
0.6710526315789473
Ticino
0.6929824561403509

---- MultiTask binary classification scores: ----

Dora
0.6008771929824561
Piemonte_Sud
0.6491228070175439
Piemonte_Nord
0.6578947368421053


##Fixed validation set

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, path):
  best_shuffles = {}

  for basin in basins :
    accuracy = []
    for i in range(0,5):
      print("Cross validation for time series of ", basin, " for shuffle ", str(i))

      ### CMI best5 features
      path_features = path
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      training_df = pd.read_csv(path_features+basin+'_'+str(i)+'_genLinCFA_best5_CMI_train.csv')
      valid_df = pd.read_csv(path_features+basin+'_'+str(i)+'_genLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', 
                                                                                      threshold = None, nopeaks = False)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std
      
      log_regr = LogisticRegression(max_iter = 500)
      log_regr.fit(training_df, target_df_train.mean_std)
      valid_score = log_regr.score(valid_df, target_df_val.mean_std)
      accuracy.append(valid_score)
      print("Train accuracy binary classification CMI best 5: ", round(log_regr.score(training_df, target_df_train.mean_std),3))
      print("Valid accuracy binary classification CMI best 5: ", round(valid_score,3))

    print("\n--- BEST SHUFFLE FOR ", basin, " : ", accuracy.index(max(accuracy)), "---\n")
    best_shuffles[basin] = accuracy.index(max(accuracy))
  return best_shuffles

In [ ]:
features_path = './GenLinCFA/temp_prec_shuffle/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './GenLinCFA/temp_prec_shuffle/')

Cross validation for time series of  Adda  for shuffle  0
Train accuracy binary classification CMI best 5:  0.574
Valid accuracy binary classification CMI best 5:  0.553
Cross validation for time series of  Adda  for shuffle  1
Train accuracy binary classification CMI best 5:  0.54
Valid accuracy binary classification CMI best 5:  0.553
Cross validation for time series of  Adda  for shuffle  2
Train accuracy binary classification CMI best 5:  0.574
Valid accuracy binary classification CMI best 5:  0.553
Cross validation for time series of  Adda  for shuffle  3
Train accuracy binary classification CMI best 5:  0.574
Valid accuracy binary classification CMI best 5:  0.553
Cross validation for time series of  Adda  for shuffle  4
Train accuracy binary classification CMI best 5:  0.574
Valid accuracy binary classification CMI best 5:  0.553

--- BEST SHUFFLE FOR  Adda  :  0 ---

Cross validation for time series of  Dora  for shuffle  0
Train accuracy binary classification CMI best 5:  0.61

In [ ]:
print("---- SingleTask binary classification scores: ----\n")
for basin in basins: 
  print(basin, " ", compute_scores_best_shuffles(features_path, [basin], True, best_shuffles))

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, cluster, False, best_shuffles)

---- SingleTask binary classification scores: ----

Adda   0.54
Dora   0.54
Emiliani1   0.72
Emiliani2   0.7
Garda_Mincio   0.71
Lambro_Olona   0.7
Oglio_Iseo   0.7
Piemonte_Nord   0.63
Piemonte_Sud   0.6
Ticino   0.67

---- MultiTask binary classification scores: ----

Emiliani1
0.7192982456140351
Emiliani2
0.7368421052631579
Garda_Mincio
0.7543859649122807

---- MultiTask binary classification scores: ----

Adda
0.6535087719298246
Lambro_Olona
0.6885964912280702
Oglio_Iseo
0.6578947368421053
Ticino
0.6885964912280702

---- MultiTask binary classification scores: ----

Dora
0.5570175438596491
Piemonte_Sud
0.6096491228070176
Piemonte_Nord
0.6403508771929824
